<a href="https://colab.research.google.com/github/Sabri2001/DLAV-2023/blob/main/CNN_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Convolutional Networks

We'll check out how to build a **convolutional network** to classify CIFAR10 images. By using weight sharing - multiple units with the same weights - convolutional layers are able to learn repeated patterns in your data. For example, a unit could learn the pattern for an eye, or a face, or lower level features like edges.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import time

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torch.utils.data as utils
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
label_names = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'
]


def plot_images(images, cls_true, cls_pred=None):
    """
    Adapted from https://github.com/Hvass-Labs/TensorFlow-Tutorials/
    """
    fig, axes = plt.subplots(3, 3)

    for i, ax in enumerate(axes.flat):
        # plot img
        ax.imshow(images[i, :, :, :], interpolation='spline16')

        # show true & predicted classes
        cls_true_name = label_names[cls_true[i]]
        if cls_pred is None:
            xlabel = "{0} ({1})".format(cls_true_name, cls_true[i])
        else:
            cls_pred_name = label_names[cls_pred[i]]
            xlabel = "True: {0}\nPred: {1}".format(
                cls_true_name, cls_pred_name
            )
        ax.set_xlabel(xlabel)
        ax.set_xticks([])
        ax.set_yticks([])

    plt.show()

In [ ]:
def get_train_valid_loader(data_dir='data',
                           batch_size=64,
                           augment=False,
                           random_seed = 1,
                           valid_size=0.02,
                           shuffle=True,
                           show_sample=False,
                           num_workers=4,
                           pin_memory=False):
    """
    Utility function for loading and returning train and valid
    multi-process iterators over the CIFAR-10 dataset. A sample
    9x9 grid of the images can be optionally displayed.
    If using CUDA, num_workers should be set to 1 and pin_memory to True.
    Params
    ------
    - data_dir: path directory to the dataset.
    - batch_size: how many samples per batch to load.
    - augment: whether to apply the data augmentation scheme
      mentioned in the paper. Only applied on the train split.
    - random_seed: fix seed for reproducibility.
    - valid_size: percentage split of the training set used for
      the validation set. Should be a float in the range [0, 1].
    - shuffle: whether to shuffle the train/validation indices.
    - show_sample: plot 9x9 sample grid of the dataset.
    - num_workers: number of subprocesses to use when loading the dataset.
    - pin_memory: whether to copy tensors into CUDA pinned memory. Set it to
      True if using GPU.
    Returns
    -------
    - train_loader: training set iterator.
    - valid_loader: validation set iterator.
    """
    error_msg = "[!] valid_size should be in the range [0, 1]."
    assert ((valid_size >= 0) and (valid_size <= 1)), error_msg

    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )

    # define transforms
    valid_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize,
    ])
    if augment:
        train_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])
    else:
        train_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])

    # load the dataset
    train_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=train_transform,
    )

    valid_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=valid_transform,
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )

    # visualize some images
    if show_sample:
        sample_loader = torch.utils.data.DataLoader(
            train_dataset, batch_size=9, shuffle=shuffle,
            num_workers=num_workers, pin_memory=pin_memory,
        )
        data_iter = iter(sample_loader)
        images, labels = next(data_iter)
        X = images.numpy().transpose([0, 2, 3, 1])
        plot_images(X, labels)

    return (train_loader, valid_loader)

trainloader, valloader = get_train_valid_loader()

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class ConvNet(nn.Module):
    def __init__(self, n_input_channels=3, n_output=10):
        super().__init__()
        ################################################################################
        # TODO:                                                                        #
        # Define 2 or more different layers of the neural network                      #
        ################################################################################
        # Trial 1: conv16k3,maxpool2,conv64k3,maxpool2,4096-512-64-10, lr0.01 -> about 42 %
        # Trial 2: conv16k3,maxpool2,conv64k3,maxpool2,4096-512-64-10, dropout 0.5, lr0.01 -> about 37 %
        # Trial 3: conv16k3,maxpool2,conv64k3,maxpool2,4096-512-64-10, dropout 0.5, lr0.01, weight_decay 0.01 -> about 37 %
        # Trial 4: conv16k3,maxpool2,conv64k3,maxpool2,4096-512-64-10, dropout 0.5, lr0.1, weight_decay 0.05 -> about 39 %
        # Trial 5: conv16k3,maxpool2,conv64k3,maxpool2,4096-512-64-10, lr0.1, weight_decay 0.05 -> about 39 %
        # Trial 5: conv64k2,maxpool2,drop,conv32k2,maxpool2,drop,2048-128-drop-10, lr0.01, weight_decay 0.01 -> about 37% %
        self.conv1 = nn.Conv2d(in_channels=n_input_channels, out_channels=64, kernel_size=2, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(p=0.5)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=2, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) 
        self.fc1 = nn.Linear(2048,128)
        self.fc2 = nn.Linear(128,n_output)
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
    
    def forward(self, x):
        ################################################################################
        # TODO:                                                                        #
        # Set up the forward pass that the input data will go through.                 #
        # A good activation function betweent the layers is a ReLu function.           #
        #                                                                              #
        # Note that the output of the last convolution layer should be flattened       #
        # before being inputted to the fully connected layer. We can flatten           #
        # Tensor `x` with `x.view`.                                                    #
        ################################################################################
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.dropout(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.dropout(x)
        x = x.view(x.size(0), -1) # flatten
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
    
    def predict(self, x):
        logits = self.forward(x)
        return F.softmax(logits)

In [ ]:
net = ConvNet()
################################################################################
# TODO:                                                                        #
# Choose an Optimizer that will be used to minimize the loss function.         #
# Choose a critera that measures the loss                                      #
################################################################################
optimizer = optim.SGD(net.parameters(), lr=0.01, weight_decay=0.01)  # Choose the optimizer you want and tune its hyperparameter
criterion = torch.nn.CrossEntropyLoss()  # the target label is NOT a one-hotted

epochs = 1
steps = 0
running_loss = 0
print_every = 20
for e in range(epochs):
    start = time.time()
    for images, labels in iter(trainloader):
        steps += 1
        ################################################################################
        # TODO:                                                                        #
        # Run the training process                                                     #
        #                                                                              #
        #                                                                              #
        ################################################################################
        optimizer.zero_grad() # don't forget!
        
        output = net(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
   
        running_loss += loss.item()
  
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################

        # QUESTION: why the heck twice...
        # ################################################################################
        # # TODO:                                                                        #
        # # Run the training process                                                     #
        # #                                                                              #
        # # HINT: Calculate the gradient and move one step further                       #
        # ################################################################################
        # pass
        # ################################################################################
        # #                              END OF YOUR CODE                                #
        # ################################################################################

        if steps % print_every == 0:
            stop = time.time()
            # Test accuracy
            accuracy = 0
            for ii, (images, labels) in enumerate(valloader):
                
                ################################################################################
                # TODO:                                                                        #
                # Calculate the accuracy                                                       #
                ################################################################################                
                predicted = net.predict(images).data
                equality = (labels == predicted.max(1)[1])
                accuracy += equality.type_as(torch.FloatTensor()).mean()
                ################################################################################
                #                              END OF YOUR CODE                                #
                ################################################################################
            
            print("Epoch: {}/{}..".format(e+1, epochs),
                  "Loss: {:.4f}..".format(running_loss/print_every),
                  "Test accuracy: {:.4f}..".format(accuracy/(ii+1)),
                  "{:.4f} s/batch".format((stop - start)/print_every)
                 )
            running_loss = 0
            start = time.time()

<ipython-input-29-da0b1cef854e>:53: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(logits)


Epoch: 1/1.. Loss: 2.3071.. Test accuracy: 0.1285.. 0.1223 s/batch
Epoch: 1/1.. Loss: 2.2776.. Test accuracy: 0.1371.. 0.1024 s/batch
Epoch: 1/1.. Loss: 2.2237.. Test accuracy: 0.1705.. 0.1005 s/batch
Epoch: 1/1.. Loss: 2.2050.. Test accuracy: 0.1732.. 0.1346 s/batch
Epoch: 1/1.. Loss: 2.1534.. Test accuracy: 0.2188.. 0.1002 s/batch
Epoch: 1/1.. Loss: 2.1457.. Test accuracy: 0.2135.. 0.1007 s/batch
Epoch: 1/1.. Loss: 2.0876.. Test accuracy: 0.2436.. 0.1433 s/batch
Epoch: 1/1.. Loss: 2.0832.. Test accuracy: 0.2432.. 0.1051 s/batch
Epoch: 1/1.. Loss: 2.0759.. Test accuracy: 0.2207.. 0.1483 s/batch
Epoch: 1/1.. Loss: 2.0647.. Test accuracy: 0.2480.. 0.1093 s/batch
Epoch: 1/1.. Loss: 2.0414.. Test accuracy: 0.2785.. 0.1415 s/batch
Epoch: 1/1.. Loss: 2.0349.. Test accuracy: 0.2770.. 0.1016 s/batch
Epoch: 1/1.. Loss: 2.0167.. Test accuracy: 0.2684.. 0.1003 s/batch
Epoch: 1/1.. Loss: 1.9798.. Test accuracy: 0.2611.. 0.1094 s/batch
Epoch: 1/1.. Loss: 1.9918.. Test accuracy: 0.2676.. 0.1346 s/b

Save best trained model.

In [ ]:
## You should be familiar with how to save a pytorch model (Make sure to save the model in your Drive)